In [1]:
%load_ext autoreload
%autoreload 2

# Import necessary modules 
import numpy as np
import pandas as pd
from itertools import product
import plotly.graph_objs as go
import plotly.express as px
from numpy.linalg import eigvalsh
from rbf_volatility_surface import RBFVolatilitySurface
from smoothness_prior import RBFQuadraticSmoothnessPrior

In [2]:
# Define the strike price list and maturity time list
strike_price_list = np.array([0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5])
maturity_time_list = np.array([0.02, 0.08, 0.17, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0])

# Create the product grid of maturity times and strike prices
product_grid = list(product(maturity_time_list, strike_price_list))
maturity_times, strike_prices = zip(*product_grid)

# Convert to arrays for further operations
maturity_times = np.array(maturity_times)
strike_prices = np.array(strike_prices)

# Variance formula for log-uniform distribution
def log_uniform_variance(a, b):
    log_term = np.log(b / a)
    var = ((b ** 2 - a ** 2) / (2 * log_term)) - ((b - a) / log_term) ** 2
    return var

# Calculate standard deviations for maturity times and strike prices
maturity_std = np.sqrt(log_uniform_variance(maturity_time_list.min(), maturity_time_list.max()))
strike_std = np.sqrt(log_uniform_variance(strike_price_list.min(), strike_price_list.max()))

n_roots = 350
smoothness_controller = 0.0001

# Initialize the RBFQuadraticSmoothnessPrior class
smoothness_prior = RBFQuadraticSmoothnessPrior(
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    maturity_std=maturity_std,
    strike_std=strike_std,
    n_roots=n_roots,
    smoothness_controller=smoothness_controller,
    random_state=0,
)

lambda_matrix = smoothness_prior.calculate_lambda_matrix()
lambda_matrix
# np.linalg.eigvalsh(lambda_matrix)

array([[ 1.15212938e-02,  2.05585801e-03,  5.90127362e-04, ...,
         6.97498512e-12,  8.64298352e-13,  1.53956719e-15],
       [ 2.05585801e-03,  4.08755930e-04,  1.52984811e-04, ...,
         1.92308189e-12,  1.49615427e-13,  3.00619154e-16],
       [ 5.90127362e-04,  1.52984811e-04,  8.71195131e-05, ...,
         1.15972959e-12,  3.88355870e-14,  1.14851151e-16],
       ...,
       [ 6.97498512e-12,  1.92308189e-12,  1.15972959e-12, ...,
         2.00000000e-06, -1.78578368e-19, -3.63062806e-18],
       [ 8.64298352e-13,  1.49615427e-13,  3.88355870e-14, ...,
        -1.78578368e-19,  2.00000000e-06,  5.51365509e-19],
       [ 1.53956719e-15,  3.00619154e-16,  1.14851151e-16, ...,
        -3.63062806e-18,  5.51365509e-19,  2.00000000e-06]])

In [3]:
np.linalg.eigvalsh(lambda_matrix)

array([1.00000000e-06, 1.00000000e-06, 1.00000000e-06, 1.00000000e-06,
       1.00000000e-06, 1.47820983e-06, 1.68119989e-06, 1.97033329e-06,
       1.98642741e-06, 1.98719812e-06, 1.99540367e-06, 1.99770697e-06,
       1.99801658e-06, 1.99840477e-06, 1.99931162e-06, 1.99969928e-06,
       1.99988935e-06, 1.99990559e-06, 1.99995225e-06, 1.99996212e-06,
       1.99999577e-06, 1.99999712e-06, 1.99999782e-06, 1.99999876e-06,
       1.99999966e-06, 1.99999988e-06, 1.99999994e-06, 1.99999997e-06,
       1.99999998e-06, 1.99999999e-06, 1.99999999e-06, 2.00000000e-06,
       2.00000000e-06, 2.00000000e-06, 2.00000000e-06, 2.00000000e-06,
       2.00000000e-06, 2.00000000e-06, 2.00000000e-06, 2.00000000e-06,
       2.00000000e-06, 2.00000000e-06, 2.00000000e-06, 2.00000000e-06,
       2.00000000e-06, 2.00000000e-06, 2.00000000e-06, 2.00000000e-06,
       2.00000000e-06, 2.00000000e-06, 2.00000000e-06, 2.00000000e-06,
       2.00000000e-06, 2.00000000e-06, 2.00000000e-06, 2.00000001e-06,
      

In [4]:
condition_number = np.linalg.cond(lambda_matrix)
print("Condition Number after regularization:", condition_number)

Condition Number after regularization: 21158.659295940328


In [5]:
pd.Series((pd.DataFrame(lambda_matrix)[pd.DataFrame(lambda_matrix) > 0].to_numpy().flatten())).dropna().describe()

count    9.782000e+03
mean     1.692801e-05
std      2.161505e-04
min      2.589871e-20
25%      9.677605e-13
50%      4.404045e-10
75%      9.326772e-08
max      1.152129e-02
dtype: float64

In [6]:
pd.Series(pd.DataFrame(lambda_matrix)[pd.DataFrame(lambda_matrix) < 0].to_numpy().flatten()).dropna().describe()

count    2.180000e+02
mean    -5.220483e-14
std      9.877641e-14
min     -6.471873e-13
25%     -5.355921e-14
50%     -4.948997e-15
75%     -4.334239e-18
max     -1.779110e-21
dtype: float64

In [30]:
np.sqrt(smoothness_prior.roots[:, np.newaxis] * smoothness_prior.roots[np.newaxis, :])

array([[1.76117167e-03, 5.28352386e-03, 8.80590262e-03, ...,
        1.51563088e+00, 1.52998429e+00, 1.54748006e+00],
       [5.28352386e-03, 1.58505981e-02, 2.64177521e-02, ...,
        4.54690026e+00, 4.58996057e+00, 4.64244797e+00],
       [8.80590262e-03, 2.64177521e-02, 4.40297345e-02, ...,
        7.57819250e+00, 7.64995993e+00, 7.73743922e+00],
       ...,
       [1.51563088e+00, 4.54690026e+00, 7.57819250e+00, ...,
        1.30432314e+03, 1.31667542e+03, 1.33173194e+03],
       [1.52998429e+00, 4.58996057e+00, 7.64995993e+00, ...,
        1.31667542e+03, 1.32914467e+03, 1.34434379e+03],
       [1.54748006e+00, 4.64244797e+00, 7.73743922e+00, ...,
        1.33173194e+03, 1.34434379e+03, 1.35971671e+03]])

In [27]:
smoothness_prior.roots

array([ 0.11758132,  1.07456201,  3.08593744,  6.41472973, 11.80718949])

In [12]:
# Define the strike price list and maturity time list
strike_price_list = np.array([0.75, 0.85, 0.9, 0.95, 1.0, 1.05, 1.1, 1.2, 1.3, 1.5])
maturity_time_list = np.array([0.02, 0.08, 0.17, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 3.0])

# Create the product grid of maturity times and strike prices
product_grid = list(product(maturity_time_list, strike_price_list))
maturity_times, strike_prices = zip(*product_grid)

# Convert to arrays for further operations
maturity_times = np.array(maturity_times)
strike_prices = np.array(strike_prices)

# Variance formula for log-uniform distribution
def log_uniform_variance(a, b):
    log_term = np.log(b / a)
    var = ((b ** 2 - a ** 2) / (2 * log_term)) - ((b - a) / log_term) ** 2
    return var

# Calculate standard deviations for maturity times and strike prices
maturity_std = np.sqrt(log_uniform_variance(maturity_time_list.min(), maturity_time_list.max()))
strike_std = np.sqrt(log_uniform_variance(strike_price_list.min(), strike_price_list.max()))

n_roots = 500
smoothness_controller = 0.0001

# Initialize the RBFQuadraticSmoothnessPrior class
smoothness_prior = RBFQuadraticSmoothnessPrior(
    maturity_times=maturity_times,
    strike_prices=strike_prices,
    maturity_std=maturity_std,
    strike_std=strike_std,
    n_roots=n_roots,
    smoothness_controller=smoothness_controller,
    random_state=0,
)

# Calculate the prior covariance matrix
prior_covariance_matrix = smoothness_prior.prior_covariance()

# Display the covariance matrix
print("Prior Covariance Matrix:")
pd.DataFrame(prior_covariance_matrix)

LinAlgError: SVD did not converge

In [3]:
# Import necessary libraries for Plotly visualization
import plotly.express as px

# Create a heatmap for the prior covariance matrix
fig1 = px.imshow(
    np.log(np.abs(prior_covariance_matrix)),
    labels=dict(x="Index", y="Index", color="Log Abs. Cov."),
    title="Smoothness Prior Log Absolute Covariance Matrix",
    color_continuous_scale="Viridis"
)

# Update layout for better visualization
fig1.update_layout(
    width=800,
    height=800,
    title_x=0.5,
    xaxis_title="Maturity and Strike Indices",
    yaxis_title="Maturity and Strike Indices",
)

# Show the figure
fig1.show()

In [4]:
# Compute the eigenvalues of the prior covariance matrix
eigenvalues = np.linalg.eigvalsh(prior_covariance_matrix)

# Sort eigenvalues by their absolute values in descending order
sorted_eigenvalues = np.sort(np.abs(eigenvalues))[::-1]

# Calculate the cumulative explained variance
cumulative_explained_variance = np.cumsum(sorted_eigenvalues) / np.sum(sorted_eigenvalues)

# Create a bar plot for the sorted eigenvalues (original values) and a line plot for cumulative explained variance
fig2 = go.Figure()

# Bar plot for eigenvalues
fig2.add_trace(
    go.Bar(
        x=np.arange(1, len(sorted_eigenvalues) + 1),
        y=np.abs(sorted_eigenvalues),
        name="Absolute Eigenvalues",
    )
)

# Line plot for cumulative explained variance
fig2.add_trace(
    go.Scatter(
        x=np.arange(1, len(cumulative_explained_variance) + 1),
        y=cumulative_explained_variance,
        name="Cumulative Explained Variance",
        yaxis="y2",
        mode="lines",
    )
)

# Add figure layout
fig2.update_layout(
    title="Absolute Eigenvalues and Cumulative Explained Variance",
    xaxis=dict(title="Index of Eigenvalues"),
    yaxis=dict(title="Absolute Eigenvalues", type="log"),
    yaxis2=dict(
        title="Cumulative Explained Variance",
        overlaying="y",
        side="right",
        showgrid=False,
    ),
    width=900,
    height=900,
    title_x=0.5,
    legend=dict(x=0.7, y=0.9),
)

# Show the figure
fig2.show()

In [5]:
5**8

390625

In [6]:
np.all(np.linalg.eigvalsh(prior_covariance_matrix) >= 0)

False

In [7]:
from numpy.linalg import pinv
np.linalg.eigvalsh(pinv(prior_covariance_matrix)/smoothness_controller**2) 

array([-3.38017377e+13, -1.80557580e+13, -4.95881328e+12, -4.38404889e+12,
       -1.59689586e+12, -2.61247316e+11, -2.50598748e+11, -1.42081573e+11,
       -1.40980702e+11, -7.76635019e+10, -6.16705702e+10, -3.13449294e+10,
       -2.60245734e+10, -1.85954004e+10, -1.51493501e+10, -6.74249257e+09,
       -1.10711422e+09, -1.08893377e+09, -1.07583560e+09, -5.76312836e+08,
       -3.93716350e+08, -3.92854800e+08, -2.73829322e+08, -1.42630108e+08,
       -1.02052382e+08, -5.34394106e+07, -5.10998180e+07, -4.94489339e+07,
       -2.93494140e+07, -1.86513282e+07, -1.84414000e+07, -8.33378523e+06,
       -6.09233610e+06, -2.94557459e+06, -2.25621602e+06, -1.30068907e+06,
       -5.96530170e+05, -5.12829783e+05, -4.31727344e+05, -2.77970100e+05,
       -2.67193784e+05, -2.44635804e+05, -2.31949881e+05, -3.92942220e+04,
       -1.99493632e+04, -1.09635020e+04, -1.09891591e+02,  3.66235501e+02,
        1.77400498e+03,  1.32855065e+04,  2.71786923e+04,  3.06038943e+04,
        6.49939632e+04,  

In [8]:
np.sort(eigenvalues)

array([-5.65209578e+08, -2.01102359e+07, -5.14580900e+06, -1.38780223e+06,
       -1.31372223e+05, -6.05739839e+04, -4.83437311e+04, -3.17516579e+04,
       -2.17175147e+04, -1.78546403e+04, -1.52899635e+04, -1.25533223e+04,
       -6.91556699e+03, -1.83380135e+03, -1.69529644e+03, -3.65007933e+02,
       -1.91743115e+02, -7.62925825e+01, -6.85297819e+01, -4.40014304e+01,
       -3.32994860e+01, -2.30782375e+01, -5.33820466e+00, -4.71020969e+00,
       -1.95547821e+00, -1.94172937e+00, -1.53975368e+00, -1.00049452e+00,
       -6.42057497e-01, -3.30961034e-01, -2.54302278e-01, -1.44339806e-01,
       -9.18823106e-02, -4.01372947e-02, -1.99190651e-02, -1.48328565e-02,
       -5.38259755e-03, -3.20393725e-03, -1.59351450e-03, -7.33499478e-04,
       -4.38022250e-04, -4.17297804e-04, -6.26789684e-05, -2.46707534e-05,
       -2.18765262e-05, -1.26629433e-05, -4.22118987e-06,  1.23217134e-06,
        7.37364529e-06,  1.08586939e-05,  1.20212740e-05,  2.51523571e-05,
        3.23695920e-05,  

In [9]:
smoothness_prior.sample_smooth_surfaces(10)

/home/hermes/scikit_learn_data/evolutionary-latent-optimization/smoothness_prior.py:215: RuntimeWarning:

covariance is not symmetric positive-semidefinite.



array([[-9.16784228e-02,  9.18725297e-03, -3.41795995e+00,
         3.38798747e+00,  2.88888364e+01,  5.80409304e+00,
        -1.67947612e+01,  1.27357687e+02,  6.54659791e+02,
         2.56675222e+01,  8.84491508e-02, -1.57110328e-02,
         3.46292797e+00, -3.42204000e+00, -2.88319561e+01,
        -5.97528335e+00,  1.69173409e+01, -1.30381601e+02,
        -6.68896003e+02, -8.38669576e+02,  1.37891653e-02,
         4.71341569e-04,  5.21229507e-02, -7.14854723e-02,
        -1.18071376e+00, -1.09797532e-01,  4.85655957e-01,
         1.26185507e+00, -2.42037424e+00, -1.39055652e+03,
        -1.86671733e-02,  4.64166078e-03,  4.96033137e-02,
        -4.83465627e-02,  7.37945914e-01,  1.11488413e-01,
         3.86277295e-01, -1.30775939e+00, -1.03384788e+01,
        -2.27511571e+03,  7.83185323e-02, -7.26443519e-02,
         1.15818889e-01, -1.87446991e-01, -8.11144563e+00,
        -1.62229995e+00,  7.19723504e+00,  2.03900901e+00,
        -1.85208580e+01,  6.39775186e+03, -2.56407574e-0